In [55]:
!rm -rf src build* my_*

In [56]:
!mkdir -p src

# Build tools

In real applications, with dozens/hundreds/thousands files, it turns tricky to recompile everything in the right order, with the right options. Also, depending on what file has been modified, not all the units needs to be recompiled, and the developer alone can hardly trace which ones. This is where the code construction tools come into play. The simple legacy one is `GNU make`. The popular current one is `CMake`.

In [57]:
%%file src/my_polar_converter.hh

#include <tuple>

std::tuple<double,double> polar_coordinates( double x, double y ) ;

Writing src/my_polar_converter.hh


In [58]:
%%file src/my_polar_converter.cc

#include "my_polar_converter.hh"
#include <cmath>

std::tuple<double,double> polar_coordinates( double x, double y ) {
  double norme = std::sqrt(x*x+y*y) ;
  double angle = std::atan(y/x) ;
  return { norme, angle } ;
}

Writing src/my_polar_converter.cc


In [59]:
%%file src/my_program.cpp

#include "my_polar_converter.hh"
#include <iostream>
#include <string>
#include <cassert>

int main( int argc, char * argv[] ) {
  assert(argc=3) ;
  double x {std::stod(argv[1])} ;
  double y {std::stod(argv[2])} ;
  auto [ n, a ] = polar_coordinates(x,y) ;
  std::cout << n << std::endl ;
  std::cout << a << std::endl ;
  return 0 ; // returns OK to the OS
}

Writing src/my_program.cpp


## GNU make

- GNU make is low level, and designed for Unix machines.
- Its command is `gmake` or `make`, and it is configured by a file `GNUmakefile`.
- A very simple example is given below.

In [60]:
!mkdir -p build.gmake

In [61]:
%%file build.gmake/GNUmakefile

SHELL = /bin/bash
CXX = g++
CXXFLAGS = -std=c++17 -O2 -Wall -Wextra -Werror

all: my_program.exe

clean:
	rm -f *.exe *.o

my_program.exe: my_program.o my_polar_converter.o
	$(CXX) $^ -o $@

%.o : ../src/%.cc
	$(CXX) $(CXXFLAGS) $< -c

%.o : ../src/%.cpp
	$(CXX) $(CXXFLAGS) $< -c

.PRECIOUS: %.o

Writing build.gmake/GNUmakefile


In [62]:
!cd build.gmake && make

g++ -std=c++17 -O2 -Wall -Wextra -Werror ../src/my_program.cpp -c
g++ -std=c++17 -O2 -Wall -Wextra -Werror ../src/my_polar_converter.cc -c
g++ my_program.o my_polar_converter.o -o my_program.exe


In [63]:
!build.gmake/my_program.exe 1. 1.

1.41421
0.785398


## CMake

- CMake is higher level.
- CMake is popular because it is working for Linux/MacOS/Windows.
- On Linux, CMake delegates the work to `make` (by default).
- CMake command is `cmake`, it is configured by the file `CMakeLists.txt`.
- A very simple example is given below.

In [64]:
!mkdir -p build.cmake

In [65]:
%%file build.cmake/CMakeLists.txt

cmake_minimum_required(VERSION 3.18)
project(my_project CXX)
    
file(GLOB HDRS ../src/*.hh)
file(GLOB SRCS ../src/*.cc)
file(GLOB PRGS ../src/*.cpp)

add_executable(my_program.exe ${SRCS} ${HDRS} ${PRGS})
target_compile_features(my_program.exe PUBLIC cxx_std_17)

Writing build.cmake/CMakeLists.txt


In [66]:
!cd build.cmake && cmake .

-- The CXX compiler identification is GNU 9.3.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /work/ClassRoom/build.cmake


In [67]:
!cd build.cmake && cmake --build .

Scanning dependencies of target my_program.exe
[ 33%] Building CXX object CMakeFiles/my_program.exe.dir/work/ClassRoom/src/my_polar_converter.cc.o
[ 66%] Building CXX object CMakeFiles/my_program.exe.dir/work/ClassRoom/src/my_program.cpp.o
[100%] Linking CXX executable my_program.exe
[100%] Built target my_program.exe


In [68]:
!build.cmake/my_program.exe 1. 1.

1.41421
0.785398


# Quizz

# Take away

- Compiling a C++ program takes a long time, that's the price to pay for a later quick execution.
- A build tool can help to:
  - find the external dependencies,
  - set the relevant compilation options,
  - (re)compile only what is needed,
  - improve portability.
  

# Questions ?

© *CNRS 2023*  
*This document was created by David Chamont. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*